In [ ]:
"""This ipynb helps show the effect to combine the patch back to the original pic"""

In [ ]:
import utils
import numpy as np
import os
from PIL import Image

In [ ]:
from prostate_dataset import ProstateDataset

In [ ]:
def instance_2_sementic(instance_mask, class_ids):
    try:
        h, w, d = instance_mask.shape
    except ValueError:
        mask = int(class_ids) * instance_mask 
        result_dict = {'ATmask': mask}
        return result_dict
    
    mask_map = {}
    for index, label in enumerate(class_ids):
        mask_map[str(label)] = np.logical_or(mask_map[str(label)], \
                                             instance_mask[:, :, index]) \
        if str(label) in mask_map.keys() else instance_mask[:, :, index]
    
    mask = np.zeros((h, w), dtype=np.int)

    for key in mask_map.keys():
        if (key != '0'):
            mask = mask + int(key) * mask_map[key] 
    result_dict = {'ATmask': mask}
    return result_dict

In [ ]:
def combine_2_whole_slide(patch_list, width_num, height_num):
    assert len(patch_list) == width_num * height_num, "Patch num doesn't match width_num * height_num!"
    row_level = []
    for i in range(height_num):
        cur_level = np.concatenate(patch_list[i * width_num : (i + 1) * width_num], axis = 1)
        row_level.append(cur_level)
    return np.concatenate(row_level, axis = 0)

In [ ]:
image_id = 318
img = []
ann = []
data_path = '/data/wenyuan/Path_R_CNN/Data_Pre_Processing'
for patch in range(16):
    dataset_dir = os.path.join(data_path, 'cedars-224')
    dataset = ProstateDataset(dataset_dir)
    image = dataset.read_image(image_id, dir_name = '/tiles_16', patch_num = patch)
    instance_mask, class_ids = dataset.read_instance_ann(image_id, dir_name = '/masks_instance_mod_16', \
                                               mode = 16, patch = patch)
    mask = instance_2_sementic(instance_mask, class_ids)
    
    img.append(image[150:450, 150:450, :])
    ann.append(mask['ATmask'][150:450, 150:450])

In [ ]:
whole_image = combine_2_whole_slide(img, 4, 4)
whole_ann = combine_2_whole_slide(ann, 4, 4)

In [ ]:
img = Image.fromarray(whole_image, "RGB")
img.show()